In [1]:
import tensorflow as tf
tf.reset_default_graph()

C:\Users\SRM\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# **Colect Data**

In [2]:
import pandas as pd
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

In [4]:
prices_df = pd.read_csv('./prices.csv')
prices_df.head(5)

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


In [5]:
prices_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851264 entries, 0 to 851263
Data columns (total 7 columns):
date      851264 non-null object
symbol    851264 non-null object
open      851264 non-null float64
close     851264 non-null float64
low       851264 non-null float64
high      851264 non-null float64
volume    851264 non-null float64
dtypes: float64(5), object(2)
memory usage: 45.5+ MB


In [6]:
prices_df.describe()

,open,close,low,high,volume
count,851264.000000,851264.000000,851264.000000,851264.000000,8.512640e+05
mean,70.836986,70.857109,70.118414,71.543476,5.415113e+06
std,83.695876,83.689686,82.877294,84.465504,1.249468e+07
min,0.850000,0.860000,0.830000,0.880000,0.000000e+00
25%,33.840000,33.849998,33.480000,34.189999,1.221500e+06
50%,52.770000,52.799999,52.230000,53.310001,2.476250e+06
75%,79.879997,79.889999,79.110001,80.610001,5.222500e+06
max,1584.439941,1578.130005,1549.939941,1600.930054,8.596434e+08


# Check all columns in the dataset


In [7]:
prices_df.columns

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')

# Drop columns date and symbol


In [8]:
prices_df.drop(['date', 'symbol'],axis=1,inplace=True)

In [9]:
prices_df.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


In [10]:
print(prices_df.isnull().sum())

open      0
close     0
low       0
high      0
volume    0
dtype: int64


### Consider only first 1000 rows in the dataset for building feature set and target set


In [11]:
prices_df_new=prices_df.head(1000)

In [12]:
y = prices_df_new['close']

In [13]:
prices_df_new = prices_df_new.drop(['close'], axis=1)

In [14]:
X = prices_df_new

In [16]:
# Split  training and testing from data
X_train, X_test, y_train, y_test = train_test_split(prices_df_new, y, test_size=0.2)
print(X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)

(800, 4) (800,)
(200, 4) (200,)


## Building the graph in tensorflow

In [17]:
# Define Input place holder
#Input features
x = tf.placeholder(shape=[None,4],dtype=tf.float32, name='x-input')

x_n = tf.layers.batch_normalization(x, training=True)

#Actual Prices
y_ = tf.placeholder(shape=[None],dtype=tf.float32, name='y-input')

In [18]:
# Define Weights and Bias

W = tf.Variable(tf.zeros(shape=[4,1]), name="Weights")
b = tf.Variable(tf.zeros(shape=[1]), name="Bias")

In [19]:
# Prediction 
y = tf.add(tf.matmul(x_n,W),b,name='output')

In [20]:
# Loss (Cost) function [Mean square error]

loss = tf.reduce_mean(tf.square(y-y_),name='Loss')

In [21]:
# GradientDescent Optimizer to minimize Loss [GradientDescentOptimizer]
train_op = tf.train.GradientDescentOptimizer(0.01).minimize(loss)

## Execute the Graph for 100 epochs and observe the loss


In [22]:
#Lets start graph Execution
sess = tf.Session()

# variables need to be initialized before we can use them
sess.run(tf.global_variables_initializer())

#how many times data need to be shown to model
training_epochs = 100

In [23]:
for epoch in range(training_epochs):
            
    #Calculate train_op and loss
    _, train_loss = sess.run([train_op, loss],feed_dict={x:X_train, 
                                                        y_:y_train})
    
    if epoch % 10 == 0:
        print ('Training loss at step: ', epoch, ' is ', train_loss)

Training loss at step:  0  is  8454.285
Training loss at step:  10  is  6900.1006
Training loss at step:  20  is  5324.4126
Training loss at step:  30  is  3779.3936
Training loss at step:  40  is  3779.1953
Training loss at step:  50  is  3779.0796
Training loss at step:  60  is  3778.9973
Training loss at step:  70  is  3778.9543
Training loss at step:  80  is  3778.932
Training loss at step:  90  is  3778.9087


## **Get the shapes and values of W and b**
Hint: Use sess.run(W) to get W.

In [24]:
sess.run(y, feed_dict={x:X_train[0:1]})

array([[67.953766]], dtype=float32)

In [25]:
sess.run(W)

array([[ 4.370232  ],
       [-3.2456212 ],
       [ 3.7319813 ],
       [ 0.44276828]], dtype=float32)

In [26]:
sess.run(b)

array([23.956633], dtype=float32)

In [27]:
print (W.shape, b.shape)

(4, 1) (1,)


In [28]:
#sess.close()

In [29]:
_, test_loss = sess.run([train_op, loss],feed_dict={x:X_test, 
                                                        y_:y_test})
    

# Find the Absolute mean square loss difference between training and testing loss.


In [30]:
Absolute_MS_Diff_loss = train_loss - test_loss
Absolute_MS_Diff_loss

1869.3948

In [31]:
X.shape

(1000, 4)

### Linear Classification using Keras
Building the simple Neural Network in Keras with one neuron in the dense hidden layer.
Use Mean square error as loss function and sgd as optimizer

In [32]:
#Initialize Sequential model
model = tf.keras.models.Sequential()

#Reshape data from 2D to 1D -> 28x28 to 784
#model.add(tf.keras.layers.Reshape(4000,),input_shape=(1000,4))

#Normalize the data
model.add(tf.keras.layers.BatchNormalization(input_shape =(4,)))

#Add Dense Layer which provides 10 Outputs after applying softmax
model.add(tf.keras.layers.Dense(1))

#Comile the model
model.compile(optimizer='sgd', loss='mse')

In [33]:
model.fit(X_train, y_train, 
          validation_data=(X_test, y_test), 
          epochs=30,
          batch_size=X_train.shape[0])

Train on 800 samples, validate on 200 samples
Epoch 1/30
800/800 [==============================] - 1s 904us/step - loss: 8565.7637 - val_loss: 1602.0028
Epoch 2/30
800/800 [==============================] - 0s 5us/step - loss: 7906.9648 - val_loss: 4046.5327
Epoch 3/30
800/800 [==============================] - 0s 4us/step - loss: 4866.7798 - val_loss: 312990.6250
Epoch 4/30
800/800 [==============================] - 0s 9us/step - loss: 810.6489 - val_loss: 812.3758
Epoch 5/30
800/800 [==============================] - 0s 10us/step - loss: 4512.5830 - val_loss: 183655.8750
Epoch 6/30
800/800 [==============================] - 0s 5us/step - loss: 1119.9304 - val_loss: 147.2439
Epoch 7/30
800/800 [==============================] - 0s 6us/step - loss: 5646.0640 - val_loss: 63650.4648
Epoch 8/30
800/800 [==============================] - 0s 7us/step - loss: 14.2144 - val_loss: 57528.3008
Epoch 9/30
800/800 [==============================] - 0s 7us/step - loss: 22.4673 - val_loss: 36246.14

## Logistic Classification using Keras

In [34]:
# Load the Iris dataset
iris_data = pd.read_csv('./Iris.csv')
iris_data.head(5)
iris_data = pd.get_dummies(iris_data,prefix=['Species'])
iris_data.head(5)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species_Iris-setosa,Species_Iris-versicolor,Species_Iris-virginica
0,1,5.1,3.5,1.4,0.2,1,0,0
1,2,4.9,3.0,1.4,0.2,1,0,0
2,3,4.7,3.2,1.3,0.2,1,0,0
3,4,4.6,3.1,1.5,0.2,1,0,0
4,5,5.0,3.6,1.4,0.2,1,0,0


In [35]:
iris_data.columns

Index(['Id', 'SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm',
       'Species_Iris-setosa', 'Species_Iris-versicolor',
       'Species_Iris-virginica'],
      dtype='object')

In [37]:
iris_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 8 columns):
Id                         150 non-null int64
SepalLengthCm              150 non-null float64
SepalWidthCm               150 non-null float64
PetalLengthCm              150 non-null float64
PetalWidthCm               150 non-null float64
Species_Iris-setosa        150 non-null uint8
Species_Iris-versicolor    150 non-null uint8
Species_Iris-virginica     150 non-null uint8
dtypes: float64(4), int64(1), uint8(3)
memory usage: 6.4 KB


In [38]:
y = iris_data[['Species_Iris-setosa','Species_Iris-versicolor', 'Species_Iris-virginica']]
X = iris_data.drop(['Species_Iris-setosa','Species_Iris-versicolor', 'Species_Iris-virginica','Id'],axis =1)

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3)
print("\nX_train:\n")
print(X_train.head())
print(X_train.shape)

print("\nX_test:\n")
print(X_test.head())
print(X_test.shape)


X_train:

     SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm
94             5.6           2.7            4.2           1.3
116            6.5           3.0            5.5           1.8
75             6.6           3.0            4.4           1.4
149            5.9           3.0            5.1           1.8
106            4.9           2.5            4.5           1.7
(105, 4)

X_test:

    SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm
38            4.4           3.0            1.3           0.2
6             4.6           3.4            1.4           0.3
34            4.9           3.1            1.5           0.1
2             4.7           3.2            1.3           0.2
43            5.0           3.5            1.6           0.6
(45, 4)


## **Model**
# Build the model with following layers: **bold text**

First dense layer with 10 neurons with input shape 4 (according to the feature set) 

Second Dense layer with 8 neurons 

Output layer with 3 neurons with softmax activation (output layer, 3 neurons as we have 3 classes) 


Use SGD and categorical_crossentropy loss

In [40]:
#Initialize Sequential model
model = tf.keras.models.Sequential()

#Reshape data from 2D to 1D -> 28x28 to 784
#model.add(tf.keras.layers.Reshape(4000,),input_shape=(1000,4))

#Normalize the data
model.add(tf.keras.layers.BatchNormalization(input_shape = (4,)))

#Add Dense Layer which provides 10 Outputs after applying softmax
model.add(tf.keras.layers.Dense(3))

#Comile the model
model.compile(optimizer='sgd', loss='mse')

In [41]:
#Add 1st hidden layer
model.add(tf.keras.layers.Dense(10, activation='relu'))

In [42]:
#Add 2nd hidden layer
model.add(tf.keras.layers.Dense(8, activation='relu'))

In [43]:
#Add OUTPUT layer
model.add(tf.keras.layers.Dense(3, activation='softmax'))

In [44]:
#Create optimizer with non-default learning rate
sgd_optimizer = tf.keras.optimizers.SGD(lr=0.03)

#Compile the model
model.compile(optimizer=sgd_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [45]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_2 (Batch (None, 4)                 16        
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 15        
_________________________________________________________________
dense_2 (Dense)              (None, 10)                40        
_________________________________________________________________
dense_3 (Dense)              (None, 8)                 88        
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 27        
Total params: 186
Trainable params: 178
Non-trainable params: 8
_________________________________________________________________


In [46]:
model.fit(X_train,y_train,          
          validation_data=(X_test,y_test),
          epochs=30,
          batch_size=32)

Train on 105 samples, validate on 45 samples
Epoch 1/30
105/105 [==============================] - 1s 11ms/step - loss: 1.0839 - acc: 0.2952 - val_loss: 1.0637 - val_acc: 0.4222
Epoch 2/30
105/105 [==============================] - 0s 133us/step - loss: 0.9880 - acc: 0.4286 - val_loss: 1.0386 - val_acc: 0.3778
Epoch 3/30
105/105 [==============================] - 0s 105us/step - loss: 0.9163 - acc: 0.6095 - val_loss: 1.0413 - val_acc: 0.2444
Epoch 4/30
105/105 [==============================] - 0s 162us/step - loss: 0.8635 - acc: 0.6476 - val_loss: 1.0650 - val_acc: 0.2444
Epoch 5/30
105/105 [==============================] - 0s 219us/step - loss: 0.8244 - acc: 0.6571 - val_loss: 1.0979 - val_acc: 0.2444
Epoch 6/30
105/105 [==============================] - 0s 495us/step - loss: 0.7966 - acc: 0.6762 - val_loss: 1.1243 - val_acc: 0.2444
Epoch 7/30
105/105 [==============================] - 0s 143us/step - loss: 0.7695 - acc: 0.6762 - val_loss: 1.1672 - val_acc: 0.2444
Epoch 8/30
105/105

# Report Accuracy of the predicted values


In [48]:
scores = model.evaluate(X_test,y_test)
scores

45/45 [==============================] - 0s 155us/step


[1.0545486715104846, 0.4222222301695082]